# Multi-task CNN Model for Attribute Prediction  

## Abstract  

本文提出了一种联合多任务学习算法，以更好地使用深度卷积神经网络（CNN）的预测图像属性。我们考虑通过多任务 CNN 模型学习二值语义属性，其中每个 CNN 将预测一个二值属性。多任务学习允许 CNN 模型同时在不同属性类别之间共享视觉知识。每个 CNN 将生成属性特定的特征表示，然后我们对这些特征应用多任务学习来预测它们的属性。在我们的多任务框架中，我们提出了一种将整体模型参数分解为潜在任务矩阵和组合矩阵的方法。此外，欠采样分类器可以利用其他分类器的共享统计信息来提高其性能。应用了属性的自然聚类，以便鼓励同一组中的属性共享更多的知识。同时，不同群体的属性通常会相互竞争，从而共享较少的知识。我们展示了我们的方法在两个流行属性数据集上的有效性。  

## I. INTRODUCTION  

使用语义属性或属性来描述对象是一种在视觉识别研究中引起了很大关注的技术。这是因为学习对象的属性可以提供有用的知识，也可以作为低级特征与高级类别之间的桥梁。各种多媒体应用程序可以从属性中受益，其中包括以下内容：知识迁移，不同目标任务之间的信息共享，多媒体内容分析和推荐，多媒体搜索和检索。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/1/1.jpg?raw=true)

通常，区分性学习方法用于学习语义属性（具有名称的属性）。图 1 显示了服装属性数据集中的两个示例，其中两个图像具有不同的属性标签。其他类型的属性，如数据驱动的属性，以无监督或弱监督的方式学习。相对属性也通过排序方法引入和学习（相对属性具有描述属性存在强度的实际值）。然而，大多数现有文献都独立地使用判别式分类器来预测低级图像特征的语义属性。考虑到它们可能在同一图像中同时发生的事实，很少有作品对对象属性之间的关系进行建模。  

像 SIFT 和 HOG 这样的低级工程特征与用于属性预测目的的各种损失目标函数结合使用。改进预测结果可以很好地指示目标任务间成功属性知识转移，例如，通过转移另一个已知类别的属性来识别目前看不见的类别。还有工作，通过对语义属训练一个朴素贝叶斯分类器，可以学习属性模型在不同类别间进行概括。然后，他们训练线性支持向量机学习非语义特征模式，并选择哪些在验证集上表现较好的属性。这种泛化的好处可以在不同的对象类别中看到，而不仅仅是跨类别内的实例。  

另一方面，深度 CNN 表现出优越的性能，在各种视觉问题中占据着最高精度基准。它也表明，CNN 能够生成强大的通用特征。CNN 通过几次卷积学习了来自原始像素的图像特征，在输入和输出之间构建了一个复杂的非线性映射。较低的卷积层捕获基本的普通措辞特征（例如，颜色斑点和边缘），并且顶层能够学习更复杂的结构（例如，车轮）。随后，人们相信人造神经网络的这种实现模仿了人脑中的视觉皮层。  

除了典型的对象识别挑战之外属性预测还引入了另外两个问题：图像多标签和基于相关性的学习。与单标签分类相比，多标签更难。它也更有可能出现在真实的场景中，此时，图像需要用来自预定义池的一组标签进行标记。例如，当用户键入他们的查询来搜索图像（例如“红色天鹅绒蛋糕”）时，引擎应该检索具有红色天鹅绒外观的真实蛋糕图像的一致结果。由于属性的巨大假设空间（例如，M 个属性需要 $2^M$），从计算角度来看这是一项困难的任务。这限制了我们以完整形式解决问题而不将其转化为多个单一分类问题的能力。特别是，应该在所有这些单个分类器之间探索相关性，以便适当分享视觉知识。多任务学习是一种有效的特征共享方法，以及分类器之间的竞争。如果任务是相关的，特别是当一个任务缺乏训练数据时，它可能从其他更适合的任务接收视觉知识。还有工作，通过使用典型的多任务逻辑回归损失函数来处理共同学习属性模型组。  

鉴于上述问题，我们针对属性预测问题提出了一个增强的多任务框架。我们将深度 CNN 特征作为我们的特征表示来学习语义属性。由于 CNN 网络结构庞大，需要强大的计算能力，因此采用以下方法：首先，如果属性数量较少，我们通过 MTL 一起训练多任务 CNN 模型，其中每个 CNN 模型都是专门用来学习一个二元属性。其次，如果属性的数量相对较大，我们对每个属性注释单独调整一个 CNN 模型以生成属性特定的特征，然后将我们提出的 MTL 框架应用于联合学习分类器来预测这些二值属性。第一种方法或多或少都适用，具体取决于可用资源（CPU/GPU 和内存）。不同属性类的视觉知识可以与所有 CNN 模型/分类器共享，以提高每个模型的性能。在现有的多任务方法中，通过引入潜在任务矩阵提出了一种灵活的特征选择方法，其中通过该潜在矩阵选择所有类别以仅共享相关视觉知识，其还可以学习局部特征。同时，也有工作有趣地利用了语义属性相关性的辅助信息。他们使用结构化的稀疏性来鼓励 groups 之间的特征竞争，并在这些 groups 中进行共享。与先前工作不同，我们引入自然分组信息并维护分解方法以通过学习局部化特征获得可共享的潜在任务矩阵，从而灵活地进行全局共享和组间竞争。此外，我们没有相互独立的预设假设，使得组在属性成员方面可能不会相互重叠。然而，由于手工制作的特征提取方法会限制性能，因此我们利用深度 CNN 模型生成更适合属性预测案例的特征。  

我们在流行的基准属性数据集上测试我们的方法：Animals with Attributes（AwA）和 Clothing Attributes Dataset。结果证明了我们的方法与标准方法相比的有效性。由于服装数据集包含少量属性，因此我们同时成功训练了我们的多任务 CNN 模型。 另外，由于 AwA 数据集包含相对较多的属性，我们首先在目标属性上训练每个 CNN 模型。然后，我们在生成的特征上应用我们的多任务框架，而没有即时的反向传播。  

我们在本文中的主要贡献总结如下：1）我们提出了一个增强的深度 CNN 结构，允许不同的 CNN 模型通过多任务分享知识; 2）我们提出了一种新的多任务方法; 自然地利用分组信息来鼓励同一组中的属性共享特征统计信息并阻止不同组中的属性共享知识。通过将模型参数分解为潜在任务矩阵和线性组合权重矩阵，我们放松对群组的任何约束，例如互斥。潜在的任务矩阵可以学习更多的局部特征，从而保持通过配置选择一些基本模式的能力。  

我们的论文的其余部分总结如下：我们首先在第二部分讨论相关工作。 第三部分介绍了除了我们的 MTL 框架细节之外，还介绍了所提出的多任务 CNN 模型。第四部分对两个已知属性数据集和结果进行了实验。最后，我们在第五节中总结该论文。  

## II. RELATED WORK  

因为这项工作主要涉及语义属性，多任务学习和深度 CNN 等主题，所以我们简要回顾一下这些方法的最新文献，包括以下内容。  

### A. Semantic Attributes  

**属性定义** 在图像中出现或消失的视觉属性。如果这个属性可以用人类语言表达，我们称之为语义属性。不同的属性可能会描述不同的图像特征，如颜色，图案和形状。最近的一些研究集中在如何通过这些中级属性来链接人类交互应用，其中人类查询表达式和查询属性短语的计算机解释之间应该发生一致的对齐。  

**全局与局部属性** 如果某个属性描述图像中的整体属性（例如“中年”男子），则属性为全局属性。通常，全局属性不涉及特定的对象部分或位置。局部属性用于描述对象的一个或多个位置，例如，'条纹驴'。这两种类型都不容易推断，因为如果分类器仅在没有空间信息（如边界框）的高级标签上进行训练，欠采样分类器的性能可能会降低。然而，有工作表明共享视觉知识可以抵消缺乏训练样本的影响。  

**相关属性** 如果属性相关且一起出现，则它们是相关的。换句话说，某些属性自然的会暗示其他属性（例如，'绿色树'和'开放天空'意味着'自然场景'），所以这种配置会对这些属性分类器施加一些等级关系。从另一个角度来说，可以从特征空间的相同部分编织属性并且可以彼此接近，例如，'黑'和'棕'属性分类器应该在特征维度空间中彼此靠近，自然属于同一组，即同一颜色组。虽然大多数现有方法训练独立分类器以预测属性，但典型统计模型（如朴素贝叶斯和结构化 SVM 模型）则用于解决该问题。  

**属性和多标签** 图像多标签只是学习为图像分配多个标签。如果问题按原样进行修改，则标签数量增加并且潜在输出标签组合变得棘手时会出现问题。 为了缓解这种情况，通过将问题分解为一组单个二进制分类器来执行常见的转换方式。预测共现属性可以看作是多标签学习。另一方面，大多数相关作品倾向于应用多任务学习来允许先验地共享或使用一些标签关系启发式。另一项研究将深度 CNN 的排序功能应用于标签分数排序。  

### B. Multi-task learning  

**为什么要用多任务学习（MTL）** MTL 最近被应用于计算机视觉问题，特别是当某些任务被抽样时。MTL 旨在强制知识共享，同时解决多个相关任务。已经证明，这种共享可以提高一些甚至所有任务的性能。  

**任务和特征相关性** 已经探索了很多共享策略;第一个考虑设计不同的方法来发现任务之间的关系，而另一个考虑另外一种方法，目的是找到所有任务中一些共同的特征结构或挖掘相关特征。最近的框架，如 Max-margin，贝叶斯和它们的联合扩展，试图发现任一个或两个任务和特征之间的相关性。尽管 Max-margin 有强大的判别能力，但贝叶斯更灵活，因此更适合进行性能推断。与这些研究相反，还有工作声称，与典型案例一样，数据的维度很高;因此，所有任务应该共享一组共同特征的假设是不合理的。他们通过同时捕获任务之间的共享特征并通过引入异常值矩阵来识别异常值来解决这些假设。在其他作品中，作者通过将模型参数分解成共享的潜在任务矩阵和线性组合矩阵来进一步放宽约束;因此，所有的任务通过这个潜在的矩阵来选择分享什么，这个矩阵可以学习更多的本地化特征。然而，在所有这些技术中，他们依靠普遍的方式来执行这种共享，通过在模型参数上应用各种正则化，如特征选择和特征竞争的结构稀疏性。  

### C. Deep CNN  

**CNN for feature learning** CNN 诞生于深度学习（DL）时代；其目标是通过使用多个非线性变换体系结构对可视化数据的高层次抽象建模。在 DL 模型中，CNN 显示出非凡的性能，特别是在图像分类和目标识别应用中。有关训练 CNN 的两个令人烦恼的问题是需要的训练样本的数量以及完全训练网络所需的时间。这意味着要有一个有效的 CNN 模型，训练数据集和时间应该足够大，以使 CNN 能够很好地执行其任务。由 CNN 学习到的特征被证明是强大的，通用的，并且比手工功能更有效。幸运的是，使用在 Imagenet 数据集上预先训练的 CNN 模型，可以更轻松地微调许多视觉数据集的各种 CNN 体系结构。  

**单标签和多标签 CNN** 使用 CNN 进行单标签预测是直观的。如前面在“属性和多标签”一节中讨论的那样，伴随多标签的挑战很多。因此，直接训练CNN 是不可行和不切实际的。然而，最近的一项工作提出了解决多标签问题的解决方案。在这项工作中，共享的 CNN 使用任意数量的图像块进行训练，这些图像块是通过一些技术提取或生成的，如二值规范梯度（BING）。与他们的方法不同，我们提出的模型具有通过多任务 CNN 模型标记具有多个标签的一个图像的本质，其通过 MTL 同时训练以允许共享视觉知识。还有工作提出了多标签的另一个方向，其中 CNN 主要用于产生现成的激活特征;然后，他们将SVM 应用于以后的分类。在我们的方法中，当属性的数量很大时，我们对许多 CNN 模型进行微调，每个模型都致力于学习属性特定的表示。这些表示被用作 MTL 后期阶段的现成特征，因为我们冻结了他们的训练，同时优化了多任务损失函数。  

**Convexity as first aids for CNN** 最近的一些工作证明凸优化可以提高高度非凸 CNN 模型的性能。提出修改 CNN 网络中的最后两层，通过将许多子模型进行线性组合，然后用凸优化族中的其他部分替换原始损失函数。他们的发现之一是 hinge 损失是在反向传播期间表现良好的优选凸函数之一。 另一项工作证实了他们的发现，即使用众所周知的 SVM squared hinge loss 在训练 CNN 之后确实提高了最终性能。通过利用这样的实验集成结果，我们采用 squared hinge loss 框架来联合优化所有分类器模型，同时应用多任务来自然地共享属性组之间的视觉知识。  

与以前的方法相反，我们提出的方法是在属性预测的深层特征上训练多任务分类器模型，并且利用可共享的潜在任务矩阵，该矩阵可以非常有用地根据属性生成输入图像的完整描述。探索这种潜在矩阵的重要性是未来感兴趣的话题。  

## III. MULTI-TASK CNN MODELS  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/1/3.jpg?raw=true)

在本节中，我们将解释提出的多任务 CNN 模型方法的细节。图 2 显示了所提出的方法的整体结构，从原始图像开始并以属性预测结束。给定一个 M 属性的词汇表，每个 CNN 模型将学习一个二值可命名属性。在所有 CNN 模型中的都进行前向置换，最后一个卷积层产生的特征将被馈送到我们的联合 MTL 损失层中。为了更清楚地说明这一点，将在损失层学习的权重参数矩阵分解为潜在任务矩阵和组合矩阵。潜在矩阵可以被看作是所有 CNN 模型之间的共享层;换句话说，潜在层用作共享的全连接层。同时，组合矩阵包含每个 CNN 模型的具体信息。它也可以被看作是在共享的潜在完全连接层之上插入的特定完全连接层。在优化联合丢失层并共享视觉知识之后，每个 CNN 模型通过反向传播优化参数。通过呈现针对多个属性注释的图像，我们迭代地训练整个结构直到收敛。  

我们采用由 Krizhevsky 提出的流行网络结构，它由 5 个卷积组成，其次是 2 个全连接层，最后是 softmax 和 loss 层。另外，在这些层之间应用了一些 pooling，normalization 和 ReLU。许多作品对这种结构的性质进行了研究和分析，并确定了一些重要的方面。例如，有工作表明，全连接层的参数占据了总网络容量的近 70％，这在训练网络时消耗了大量的工作量。然而，考虑到在“good-but-fast”和“perfect-butslow”之间的选择，当移除全连接的层时，性能会略有下降。因为我们的模型需要多于一个 CNN 模型，所以我们移除最后一个全连接层，因为我们用我们自己的联合 MTL 目标损失替代这些层，具体取决于所学的权重参数矩阵。  

在下面的小节中，我们将演示共享的潜在任务矩阵（也可以将其视为多任务 CNN 模型方法中的共享层）。然后，我们将展示如何进行特征分享和竞争。接下来，我们介绍我们的公式，我们用它来解决属性预测问题。最后，描述了用于训练整个多任务 CNN 模型网络的总体优化程序。在本文的其余部分，我们将使用 task/CNN 模型作为分类器的可互换含义，因为在所有情况下，我们都使用相同的基础 MTL 框架。  

### A. Sharing the Latent Task Matrix in MTL  

给定 M 个语义属性，目标是为它们中的每一个学习二元线性分类器。每个分类器或任务都有模型参数，用 $w^M$ 表示，并专门用于预测相应的属性。W 是分类器总权重矩阵，其也可以被认为是 softmax 权重矩阵，但是是所有 CNN softmax 层的叠加。给定 N 个训练图像，它们中的每个都具有 M 维的标签矢量 Y，使得 $Y_m^i = 1 or Y_m^i = -1$，表示图像 i 是否有属性 m。假设每个 CNN 模型中最后一个卷积层的输出形成我们的输入特征向量，这样每个CNN 模型将生成一个属性特定的训练池。 因此，我们将从所有 CNN 模型中汇总 $X_M^N$ 训练示例。  

我们的假设来自之前的工作，其中每个分类器可以从许多共享的潜在任务和这些任务的线性组合中重建。通过这种分解，CNN 模型可以共享相似的视觉模式，并从潜在层进行灵活的选择，从而学习更多的局部化特征。我们将 L 表示为这个潜在任务矩阵，而 $s^m$ 是一个属性特定的线性组合列向量。总的来说，我们有所有属性分类器的 S 个线性组合矩阵。  

现在，我们想将 W 分解成两个矩阵 L 和 S，因为我们假设 W 是共享 L 潜在矩阵和组合矩阵 S 的乘积，W = LS。 为了更具体地说明每个属性分类器，可以通过将 L 与相应的 $s^m$ 向量相乘来形成权重参数向量：  

$w^m = Ls^m$

其中 m 是第 m 个属性的索引，m = {1,2,3 ... M}。  

给定 CNN 模型，我们的目标是学习矩阵 W，它是通过堆叠每个 CNN 的 softmax 层的参数矩阵而形成的。我们模型背后的主要思想是将这个权重矩阵 W 分解成两个矩阵 L 和 S，其中 L 潜在矩阵是所有 CNN 模型之间的共享层，S 是组合矩阵，每列对应一个 CNN 分类层。  

通过这种分解，每个 CNN 可以通过潜在矩阵 L 与其他 CNN 模型共享视觉模式，并且所有 CNN 模型可以在训练阶段一起协作以优化该共享层。每个CNN预测图像是否包含相应的属性。通过多任务学习共享层的好处是即使其训练样本不够,每个 CNN 都可以通过学习其他 CNN 模型来利用视觉知识。  

### B. Feature Sharing and Competition in MTL  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/1/3.jpg?raw=true)

根据文献中提出的语义属性分组思想，将组细节用作区分性辅助信息。组成员的特性有助于促进哪些属性分类器被鼓励共享更多的视觉知识。同时，不同的群体倾向于相互竞争，分享更少的知识。表 1 显示了动物属性（AwA）数据集中的一些组信息示例。因为属性是自然分组的，如果它们属于相同的组，我们通过鼓励属性分享更多的分组信息来编码分组信息，并且如果它们属于不同的组，则彼此竞争。我们的属性组信息如表 2 所示。  

假设我们有 M 个属性和 G 个组，其中每个组包含可变数量的属性，例如 g1 包含 [a1，a2，...，aC]，如图 2 的左侧所示，每个组可以有最多 M 个属性。我们对组内属性没有限制。即使两个组具有相同的属性，通过学习更多本地化功能的能力，潜在层配置可以缓解重叠组的影响。然而，在我们的实验中，我们依赖于数据集中提供的现有分组信息，显然这些组是相互排斥的（一个属性只能在一个组中看到）。通常，解决重叠组问题需要一些 interior-point 方法，这是一种在 second-order cone programming，在计算上是昂贵的。因此，在我们的 MTL 框架中，我们利用组的丰富信息，并采用灵活的分解来通过潜在矩阵学习不同的局部化特征。  

正则化是我们用于平衡组内属性分类的特征共享和组间属性分类之间的特征竞争的关键点。这个想法是，当使用 L1 范数时，它将促进 W 的 行/特征和列/的稀疏性。稀疏性对行的影响将产生任务之间的竞争；同时，列上的稀疏效应会产生稀疏向量。此外，当使用 L21 $\sum_{m=1}^M|| w_d||_2$ 范数时，其中 D 是特征维空间，在我们的例子中，因为它是从前一层提取的，D 是    4096。这可以看作是将 L1 范数应用到 L21 范数的输出上，强制任务只选择可由其他任务共享的维度，以鼓励特征共享。作为中间解决方案，如果在应用 L21 规范时使用语义组信息，则 group 间就有竞争；同时，可以在每个组内部应用共享。  

在我们的框架中，我们通过调整 L21 正则化来鼓励组内特征共享和组间特征竞争。  

### C. Formulations of the Multi-task CNN model  

鉴于以上关于分解 W 的讨论，并通过将正则化与分组信息一起用于更好的特征竞争和共享，我们提出了以下目标函数：  

$min_{L,S} = \sum_{m=1}^{M} \sum_{i=1}^{N_m} 1/2 [max(0,1-Y_m^i(Ls^m)^TX_n^i)]^2 + \eta \sum_{k=1}^{K} \sum_{g=1}^{G} \parallel s_k^g \parallel_2 + \gamma \parallel L \parallel_1 + \gamma \parallel L \parallel_F^2$

除了我们的额外正则化之外，这是典型的 squared hinge loss function。对于第 m 个属性类别，我们将其模型参数表示为 $Ls^m$，相应的训练数据为 $(X_m^i,Y_m^i)$。  

因此，每个 CNN 都有责任学习更好的输入特征表示。在后面的测试中，输入图像将被输入到所有的 CNN 模型中以生成不同的输入特征向量; 那么将应用相应的分类器权重向量来产生属性预测。  

每个 CNN 模型中的底层都按照前面中提出的网络结构相同的方式定义。如图 2 所示，每个 CNN 块都有几个隐藏层，主要是 5 个卷积。我们用我们提出的MTL  squared hinge loss 层来替换最后 2 个全连接层，softmax 和 loss。尽管如此，当属性数量很大时，我们冻结底层的训练并优化多任务损失函数以使用 CNN 模型生成的输出来预测属性。  

### D. Optimization Steps  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/1/4.jpg?raw=true)

回想一下，在 M 个 CNN 模型的训练过程中，每个模型都负责预测单个属性。我们的目标是通过优化多任务目标函数来强化所有 CNN 模型之间的视觉知识共享。W 的优化组件将作为最后两个全连接层。L 部分是所有 CNN 模型之间的共享层。通过利用来自其他属性分类器的共享视觉知识来提高每个 CNN 的泛化能力。随机梯度下降（SGD）优化器的负担只集中在底层图像的训练方面，如果它们在训练中不冻结，那么每个 CNN 都可以提供强大的图像特征表示。  

优化所提出的代价函数是复杂的，因为它在 L 或 S 上不是共凸的。而且，当 L 固定时，优化问题按照 S 来描述，并描述如下：  

$min_{L,S} = \sum_{m=1}^{M} \sum_{i=1}^{N_m} 1/2 [max(0,1-Y_m^i(Ls^m)^TX_n^i)]^2 + \eta \sum_{k=1}^{K} \sum_{g=1}^{G} \parallel s_k^g \parallel_2$

在训练时期，所有 CNN 模型的正向传播将生成多任务损失层的输入。在使用所提出的算法 1 对方程进行优化之后，输出是整体模型权重矩阵 W，其中 W 中的每列将专用于其特定的相应 CNN 模型，并且在相对于其输入的梯度。W 使用 L 和 S 的最优解进行重构。  

## IV. EXPERIMENTS AND RESULTS  

### A. Datasets and Grouping  

我们在两个数据集上进行实验：  

**服装属性数据集** 该数据集包含 1856 个图像和 23 个二元属性，以及 3 个多类属性。在图像级别上提供了真实标签，并且每个图像都针对所有属性进行了标注。我们忽略多类属性，因为我们只对二元属性感兴趣。这种服装属性背后的目的是提供更好的服装识别。我们训练多任务 CNN 模型以预测此数据集中的属性。因为在这个数据集中没有建议分组信息。在表 II 中，我们将在服装属性数据集中显示我们的属性分组细节。  

**AwA数据集** AwA 数据集的目的是应用迁移学习和 zero-shot 识别。它由 50 个动物类的 30475 个图像组成。提供类/属性矩阵；因此，在动物类别级别进行标注。它为每个类提供 85 个二元属性。该数据集有 9 组：颜色，纹理，形状，活动，行为，营养，性格，栖息地和身体部位；表 I 显示了其中一些组中的一些属性。  

### B. Attribute Prediction Accuracy  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/1/5.jpg?raw=true)

我们对这两个数据集进行了几次实验。对于服装数据集，我们同时训练多个 CNN 模型。根据数据集中提供的标签计算属性预测的准确性。在表 3 中 S-extract 是指一个简单的坐标，我们直接使用 CNN 预训练模型进行特征提取，然后我们训练单个 SVM 任务用于属性预测；同时，在 M-extract，我们在相同 CNN 提取的特征上训练我们的 MTL 框架。S-CNN 是指单任务 CNN，我们对 CNN 的各个模型进行了细化以预测每个属性，而 M-CNN 是指我们的 MTL 框架，而不对组信息进行编码，而 MG-CNN 是我们的整体 MTL 框架与组编码和整体训练 CNN 模型与我们的框架在一起。CF 是指没有姿态基线的组合特征模型，而 CRF 是指由之前工作提出的最先进的方法。我们的模型胜过了 CRF 中的最新成果。但我们注意到，与我们在 AwA 中的结果相比，单个 CNN 任务模型的整体改进余量相对较小（参见表IV）。这是因为精度结果已经很高，因此难以进一步改进。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/1/6.jpg?raw=true)

我们对 AwA 数据集进行了另一个实验。我们对每个属性单独调整了单个 CNN 模型。稍后，在输入图像的情况下，我们使用这些学习模型来提取属性特定的特征。换句话说，我们冻结了所有 CNN 模型中底层的训练，并且只在训练我们的多任务损失层时进行了详细说明。这是由于 AwA 数据集中有大量的属性。我们注意到，微调阶段不会增加很多实际的差异，并且是一个非常耗时的过程，这可能是由于 AwA 和 Image-net 数据集具有大约 17 个对象类别的重叠；另一项工作也探讨了这个问题，他们甚至还训练 CNN 模型来对 AwA 数据集上的对象进行分类；然而，他们注意到直接在 Imagenet 数据集上使用预先训练的 CNN 模型或在 AwA 上微调模型将在两种情况下给出相同的属性预测结果。但是，我们的 MTL 框架大大超过了单个任务;表 IV 显示了我们的方法与其他标准方法相比的性能，其中预测准确性是以所有属性的平均值为基础的。与先前大约 75％ 的最先进结果相比，我们训练的 MTL CNN 模型在性能上远远优于它。此外，表V 显示了应用我们的多任务框架之前和之后，各组属性的平均精度（组级精度）的精度结果。在我们的实验中，再次使用 Imagenet 上预先训练好的模型进行初始化。图 3 还显示了一些来自单任务分类器的错误分类测试样本，我们的多任务分类器正确分类。  

### C. Implementation Details  

**CNN模型训练** 我们把每个 CNN 模型迭代 100 次，尽管大多数模型大约50次时收敛。我们使用 Imagenet 上预先训练好的网络初始化每个 CNN 模型并在目标属性注释上对其进行微调。我们将输入图像标准化为 256x256，并从中减去平均值。为了在服装数据集上训练我们的模型，我们使用了数据增强策略。  

**多任务优化** 我们对 W 执行奇异值分解（SVD）以获得 L 的初始化；同时，S 被随机初始化。W 的初始化通过堆叠来自所有预先训练的 CNN 模型的最后全连接层来完成。其他模型参数值可以通过实验选择，也可以遵循典型启发式和策略。潜在任务数被设置为最大可能的特征维数，因为属性预测的应用对于任何细微的细粒度细节都非常关键；因此，任何由 SVD 引起的严重信息损失都可能大大降低性能。因此，在我们的实验中，潜在任务的数量被设置为2048。  

此外，我们将 CNN 模型中的权重衰减设置为 0.0005；动量设定为 0.9，学习率初始化为 0.01，在整个训练过程中手动减少。  

我们在两个 NVIDIA TK40 16GB GPU 上进行实验；服装数据集包括 CNN 部分和 MTL 部分训练在内的总体训练时间大约为1.5天，并且包括来自所有 CNN 模型的特征提取的测试时间大约为 50 分钟从模型中逐一提取，而不是并行提取，每个模型中提取特征所需的时间约为 1.5 分钟/ 1000 张图像）；如果添加更多属性 CNN 模型，时间将最终增加。对于 AwA 数据集，我们将它的训练图像分成几组（每组包含 3000 个图像，我们有 8 组，5 个用于训练，3 个用于测试）。总的来说，训练时间大约需要2周（但是，因为我们注意到，主要的准确率增加主要来自训练我们的 MLT 框架，而不是来自 CNN 微调，我们重新进行了实验并冻结了底层并且依赖于正如我们之前讨论的那样，对 MTL 层进行训练；但在第二个实验中，我们节省了大量的训练时间，因为在一次训练中，我们的 MTL 框架内的所有 CNN 模型上的最后两层完成训练仅需大约 13 小时；在所有剩余的集合上，大约需要 2.5 天才能完成训练。  

## V. CONCLUSION  

在本文中，我们介绍一种增强的多任务学习方法，以更好地预测语义二元属性。我们提出了一个多任务 CNN 模型，以允许在任务之间共享视觉知识。我们在我们的 MTL 框架中编码语义组信息，以鼓励同一组中的属性之间进行更多共享。我们还提出将模型参数分解为潜在任务矩阵和线性组合矩阵。潜在任务矩阵可以有效地学习局部特征模式，任何欠采样分类器将通过利用这个可共享的潜在层来更好地推广。这种潜在任务矩阵的重要性是未来感兴趣的话题。具体而言，我们想探索潜在任务矩阵分解的潜力，足以提供足够的信息来根据语义属性或潜在属性生成对输入图像的有效描述。我们对两个属性基准数据集的实验表明，我们学习的多任务 CNN 分类器很容易胜过以前的单任务分类器。

# Generative Adversarial Models for People Attribute Recognition in Surveillance  

## Abstract  

本文中，我们提出了一种在监视环境中检测人物属性（例如性别，种族，服装等）的深层次的体系结构。我们建议通过深度卷积生成对抗网络（DCGAN）增强图像来处理在监视场景中经常出现的低分辨率和遮挡问题。实验表明，通过结合我们的生成重建和深层属性分类网络，可以有效地提取属性，即使分辨率很差并且存在强烈的遮挡时，也可以达到整个人物的80％。  

## 1. Introduction  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/2/1.jpg?raw=true)

监控摄像头在全球大部分城市迅速传播。计算机视觉中的监视任务主要集中在跟踪和检测目标，而目标通常用视觉外观进行描述。尽管如此，为了更好地了解情况，捕捉尽可能多的人的特征已变得越来越重要。文献中称为属性识别的任务在于通过安全摄像头检测人的属性（如年龄，性别等）和物品（背包，行李等）。虽然这项任务从捕获性别，年龄和种族的脸部识别受到了有利的冲击，但很少有作品关注整个人的身体。其中，其中大多数人认为人们总处于没有遮挡且清晰的场景中，但是在监控场景中并不是这样的。事实上，通常具有较高视野的监控摄像机受到分辨率问题和人员遮挡的严重影响，如图 1 所示。  

在这项工作中，我们提出了一种属性识别方法，通过利用深度生的网络方法明确地处理分辨率和遮挡问题。我们的方法由三个网络组成。第一类将人物全身图像进行属性识别。其他人则致力于通过提高分辨率来增强输入图像，并试图通过生成卷积方法从遮挡中重建图像。据我们所知，这是第一个通过明确处理这两个问题来考虑监视任务的工作。  

## 2. Related Work  

早期关于属性识别的研究通常通过为每个属性独立训练不同的分类器来处理属性。最近，卷积神经网络（CNN）使研究人员能够挖掘属性之间的关系，并且由于其先进的性能而成为大规模物体识别问题的首选。在 CNN 上有大量的工作，比如在有遮挡和低分辨率情况下进行面部性别分类任务，或预测野外面部的面部特征。许多其他作品提出了不同的方法来实现属性分类，如在无约束的环境中的性别，微笑和年龄。但是，这些技术只涉及面部图像，并不适用于监视任务。此外还有工作解决了基于服装属性来描述人的问题。尽管如此，我们的工作将整个人视为一个整体，并不仅仅关注服装分类。最近一些依靠全身图像推断人类属性的工作是属性卷积网络（ACN）和基于深度学习的 MultiAttribute 联合识别模型（DeepMAR）。ACN 通过联合训练的整体 CNN 模型共同学习不同属性，而 DeepMAR 利用对象拓扑中的先验知识进行属性识别。还有属性分类是通过结合 part-based 模型和深度学习通过训练姿态规范化的 CNN 来完成的。此外，MLCNN 将人体分为 15 个部分，并为每个人分别训练 CNN，而 DeepMAR\* 将整个身体分为三部分，分别对应于行人的头部部分，上身和下身。此外，还有工作通过改进深层次上下文中的 part-based 的方法来解决属性识别问题。尽管如此，这些方法的大部分依赖于高分辨率的图像，并不包含遮挡问题。最近的关于图像超分辨率的研究利用了生成对抗网络（GAN），更精确地说是深度卷积生成对抗网络（DCGAN），以便从低分辨率网络开始生成高分辨率图像。  

## 3. Method  

这项工作的贡献包括三个网络：性能最好的基于 ResNet 的 part-based 人体属性分类基础架构，一种生成模型，旨在重建遮挡图像中人的身体缺失部分 以及能够在低分辨率下提高图像分辨率的第二种生成模型。  

### 3.1. Attribute Classification Network  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/2/2.jpg?raw=true)

所提出的人类属性分类方法受以前 part-based 作品的启发，因此能够从不同部分学习姿态归一化深度特征表示。通过将神经网络与 part-based 方法的能力混合在一起，在面对由姿势和视点的影响支配的不受约束的图像时赋予了鲁棒性。受到其他工作启发，我们提出将输入图像 I 分解成对应于全身 b 和一系列局部区域 $\{p \in P\}$（图 2 中的输入）。我们选择三部分：行人的头肩部分，上身和下身。然后，这四个块通过在 ImageNet 分类上预训练的ResNet50 网络，以获得四个区域的卷积特征。请注意，为了实现这一目标，我们将 ResNet50 最后的 7×7 的 average pooling 替换为 global average pooling。这使得我们可以为网络提供尺寸小于 227 的图像。在计算特征映射后，我们分出两个属性评分路径。在第一条路上，我们使用全身特征图来获得基于整个人的预测分数。我们将第二部分的 part-based 技术应用于计算每个图像分区的预测分数（图 2 中的分数），然后是最大分数操作，旨在为每个属性选择最具描述性的部分。此操作是必需的，因为人的属性通常存在于特定的身体区域中（例如，“帽子”在头肩部分中）。通过将全身评分添加到部件评分来执行最终属性预测：  

${Score}_i(I) = {Score}_i(b) + {Score}_i(P) = w_{i,b}^T \cdot \phi(b) + {max}_{p \in P} w_{i,p}^T \cdot \phi(p)$

其中 $w_i$ 是不同区域第 i 个属性的评分权重，而 $\phi(\cdot)$ 是不同区域的特征图。  

图 2 中描绘的整个网络使用具有不同权重的加权二值交叉熵损失对每个属性进行训练。使用加权损失的需要源于数据集中属性类的分布可能不平衡的事实：  

${Loss}_C = - \sum_{i=1}^A \frac {1}{2r_i} \cdot y_i \cdot log(\hat y_i) + \frac {1}{2(1-r_i)} \cdot (1 - y_i) \cdot log(1-\hat y_i)$

其中 A 是属性的总数，y 是真实标签，$\hat y$ 是预测标签。对于每个属性，r 是拥有该属性的样本数与样本总数之间的比率。  

### 3.2. Reconstruction Network  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/2/3.jpg?raw=true)

我们的第二个目标是使先前的体系结构具有强大的抗遮挡能力，以整合一个能够消除遮挡的系统，并用可能属于遮挡人员的身体部位替换它们。在最坏的情况下，即使被替换的身体部位不能反映人的真实属性，重建仍然有助于分类：通过去除遮挡，我们生成的图像仅包含没有可能导致错误分类的噪声的对象。例如，包含被另一个人遮挡的人的图像可以诱导网络对前景中不是图像主题的人的属性进行分类。为了实现这一点，我们训练一个能够从遮挡输入图像 $I^O$ 估计重建图像 $I^R$ 的生成函数 $G^R$。在训练期间，通过人工部分地将图像 I 与另一图像重叠来获得 $I^O$。为了实现我们的目标，我们训练一个具有参数 $\theta_g$ 的 生成网络作为一个前馈CNN $ G_{\theta_g}^R$。对于 N 个训练图像：  

$\hat \theta_g = arg {min}_{\theta_g} \frac {1}{N} \sum_{n=1}{N} {loss}_R(G_{\theta_g}^R(I_n^O),I_n)$

这里 $\theta_g$ 是通过最小化本节末尾描述的损失函数 ${loss}_R$ 得到的。  

我们进一步定义一个参数为 $\theta_d$ 的鉴别器网络 $D_{\theta_d}^R$，我们与 $G_{\theta_g}^R$ 一起训练以解决对抗最小最大问题：  

${min}_{G^R}{max}_{D^R} \ \Bbb E_{I ∼ p_{data}(I)} [\log D^R(I)] + \Bbb E_{I^O ∼ p_{gen}(I^O)} [\log 1 - D^R(G^R(I^O))]$

鉴别器 $D^R$ 的目的是将生成的图像与真实图像区分开来，同时生成器 $G^R$ 被训练以愚弄鉴别器。通过这种方法，我们获得了一个生成器模型，这个生成器的输出结果与没有遮挡的图像相似且判别器也不能区分。我们提出的生成器架构如图 3 所示。具体来说，在编码器中，我们使用四个步长为2的 strided 卷积层（图 3 中的 SConvs）来减少每次特征数量增加一倍时的图像分辨率，解码使用四个也称为 fractionally strided 卷积的转置卷积层（TConvs），以在每次特征数量减半时提高分辨率。编码阶段使用 Leaky ReLU 作为激活函数，解码阶段使用 ReLU。我们在激活之前采用batchnormalization 层（除了最后一个Conv）。鉴别器结构与生成器的编码器相似，除了滤波器的数量从 128 到 1024 之外。得到的 1024 个特征映射之后是一个 S 形激活函数，以获得对分类有用的概率。除了第一层我们在每次 Leaky ReLU 激活之前使用 batchnormalization。  

损失函数 LossR 的定义是我们生成网络有效性的基础，我们提出了由两个分量的加权组合构成的损失：  

${Loss}_R = {Loss}_{SSE} + {Loss}_{gen}$

这里 ${Loss}_{SSE}$ 是基于预测误差平方和（SSE）的重建损失，其使得生成器预测像素类似于目标图像的图像。应用平均池化层在缩小的生成和目标图像之间计算像素方式的 SSE。这是因为我们想要避免 MSE 和 SSE 训练自动编码器遭受的标准“模糊”影响。 在我们的实验中，我们使用 $\lambda = 10^{-1}$。  

第二个组件 ${Loss}_{gen}$ 是生成器 $G^R$ 的实际对抗性损失，它鼓励网络生成类人图像子空间中的感知良好的结果，损失定义如下：  

${Loss}_{gen} = \sum_{i=1}^N \log (1-D^R(G^R(I^O)))$

$D^R(G^R(I^O))$ 是鉴别器将生成的图像 $G^R(I^O)$ 标记为真实图像的概率。  

### 3.3. Super Resolution Network  

我们的最后一个目标是将我们的系统与能够提高质量较差图像分辨率的网络相集成。通过训练能够从低分辨率输入图像 $I^L$ 估计高分辨率图像 $I^H$ 的另一生成函数 $G^S$ 来完成该任务。在训练期间，通过以因子 r = 4 执行简单的下采样操作从原始图像 I 获得 $I^L$。为了实现我们的目标，就像我们为 $G^R$ 做的那样，我们将生成网络作为前馈 CNN 进行训练。至于重构网络，我们用我们定义 $D^R$ 的相同方式定义鉴别器 $D^S$。对于 $D^S$，我们使用了 $D^R$ 中使用的相同体系结构。两种模型之间的架构差异存在于层数中：在超分辨率网络中，我们使用三个分步卷积，分别具有 256, 512 和 1024 个特征以及五个转置卷积,分别具有512,256,256,128,128个特征。其动机是 $G^S$ 中的输入图像相对于 $G^R$ 中的输入图像小两倍。此外，在 $D^S$ 中，我们使用了五个步进卷积，滤波器的数量从 128 到 2048 增加了 2 倍。最后，我们将损失分量权重 $\lambda$ 设置为 1。  

## 4. Experiments  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/2/4.jpg?raw=true)

我们使用新的 RAP 数据集进行实验，这是一个非常丰富的带有 41,585 个行人样本的标注数据集，每个样本都标注了 72 个属性以及视点，遮挡和身体部位信息。正如前面论文所建议的那样，我们用五个度量衡量表现：平均准确率（mA），准确率，精度，召回率 和 F1 得分。我们进行三种类型的实验。首先，我们比较评估了属性分类网络的性能，并将结果与其他深度 SoA 方法进行比较。其次，我们用带有遮挡的数据集（图 4 中的 occRAP 第 2 行）来破坏数据集，并测试了我们的 Reconstruction Network 组合所带来的好处。第三，我们通过降低分辨率来破坏数据集（图 4 中的 lowRAP 第三行）并评估我们超高分辨率网络的贡献。最终，我们提出了一个完整的分类过程，将所有三个网络在低分辨率和遮挡图像上组合在一起。 occRAP 数据集是通过随机重叠 RAP 图像生成的，以便人工再现遮挡。请注意，在我们的实验中，我们仅将注意力集中在一种类型的遮挡上：覆盖图像底部的遮挡，其中遮挡部分已经从 50％ 到 80％ 的遮挡率随机采样。相反，lowRap 是通过从原始 RAP 图像中以因子 4 执行简单的下采样获得的。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/2/5.jpg?raw=true)

**属性分类** 我们在 RAP 数据集上进行了 5 次随机划分数据集的实验。对于每个分割，总共 33,268 张图像用于训练，其余 8,317 张图像用于测试。由于 RAP 中属性的不平衡分布，我们选择了数据集中正例比例的高于 0.01 的 50 个属性。对于每个图像，我们还添加一个属性，对应于我们感兴趣的遮挡（遮挡属性）。对于每个小批次，我们将图像大小调整为 320×128 的固定尺寸。为了在三个部分中分割图形，我们将高度分成 10 个块，并挑选顶部 4 作为头肩部分，第三个到上第七个为身部分，下身部分是第六个到第十个。该网络使用随机梯度下降进行训练，学习率为 $10^{-5}$，学习速率衰减为 $10^{-5}$，动量为 0.9。我们每个 mini-batch 中有8个图像。表 1 显示了 RAP 数据集的结果，其中我们在 51 个属性上比较了我们的 baseline 和其他 state-of-theart 的方法，可以看出，我们的网络在精度方面表现优异，在精度和召回相关指标方面都具有竞争力。这主要是由于采用了图像固定的身体部位。此外，区域评分最大化允许为每个单独属性选择最可靠的分数，从而提高分类准确性。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/2/6.jpg?raw=true)

**重建** 我们用 occRAP 训练集训练了我们的重建网络，同时为网络提供了与输入相关的原始未遮挡图像，以便计算 LossSSE。为了优化，我们使用了Adam，$\beta_1 = 0.5$，学习率为 0.002。我们用 K = 1 来更新鉴别器和生成器网络。此外，其目的是量化遮挡图像在分类任务中的影响。我们像分类网络中输入了 occRAP 的测试集图像，得到了表 2 中的结果，在图 5 中进行可视化。其次，为了是去除遮挡，我们重复了重建网络的操作。在同一张表中报告的结果显示出显著的改善。从结果可以看出，我们的重建网络提供了一个合理的猜测，即被遮挡人的外表能够从可见部分获知可能有用的图像重建。  

**分辨率** 用于训练超分辨率网络的数据集是 lowRAP。我们使用与输入相关联的原始大小图像来训练网络，以便计算 LossSSE。我们采用了与重构网络相同的优化器和相同的 K 值。为了评估低分辨率如何影响属性分类的性能，在 4 倍双线性上采样之后，我们将 lowRAP 测试集输入属性分类网络。随后，我们在属性分类之前通过超分辨率网络传递图像。从表 2 中可以看出，采用超分辨率网络之后可以保存更多的信息。  

**完整模型** 我们的最终实验包括测试我们所有的网络，以建立一个能够检测损坏图像的系统，并在可能的情况下进行恢复操作。为了达到这个目的，我们提出一个简单的算法，只有当输入图像小于网络输入时，输入才通过分辨率网络，然后通过分类网络，并且如果正向触发了遮挡属性，则图像通过重建网络传递，重建后的图像最终再次输入分类网络输出最终分数。该测试是在 occRAP 和 lowRAP 测试集的合并上执行的。 表 2 强调了这条管道获得的改进结果。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/attribute/2/7.jpg?raw=true)

## 5. Conclusions  

在这项工作中，我们介绍了使用深度生成网络对人物属性进行图像增强分类。我们的生成网络旨在克服监视场景中的两个常见问题，即分辨率和遮挡。实验表明，在将图像馈送到属性分类网络之前增强图像可以改善结果。在进一步的工作中，我们将探讨单一端到端模型中网络的融合，通过在测试时查看图像可以自动选择激活哪个增强网络。我们发现这种工作方式可以促进关监视环境下属性分类问题的研究。

## 参考文献  

- [Multi-task CNN Model for Attribute Prediction ](https://arxiv.org/abs/1601.00400)  
- [Generative Adversarial Models for People Attribute Recognition in Surveillance](https://arxiv.org/abs/1707.02240)